### Assistive Robotic Arm for Paralyzed Individuals Using Facial Gestures

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import serial
import time

# Initialize serial communication with Arduino
def initialize_serial(port, baud_rate=9600):
    try:
        arduino = serial.Serial(port, baud_rate)
        time.sleep(2)  # Allow time for connection to initialize
        print("Serial connection established.")
        return arduino
    except Exception as e:
        print(f"Error initializing serial: {e}")
        return None

# Initialize Mediapipe FaceMesh
def initialize_face_mesh():
    mp_face_mesh = mp.solutions.face_mesh
    return mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Function to calculate gaze direction
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to detect blinking
def is_blinking(eye_landmarks, threshold=0.2):
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    blink_ratio = vertical_distance / horizontal_distance
    return blink_ratio < threshold

# Function to detect mouth status
def detect_mouth_status(landmarks):
    upper_lip_y = landmarks.landmark[61].y
    lower_lip_y = landmarks.landmark[17].y
    lip_distance = abs(upper_lip_y - lower_lip_y)
    
    left_eye = landmarks.landmark[33]
    right_eye = landmarks.landmark[263]
    eye_distance = abs(left_eye.x - right_eye.x)
    
    threshold = 0.025 + (eye_distance * 0.1)
    return "Mouth Open" if lip_distance > threshold else "Mouth Closed"

# Function to calculate eyebrow movement
def calculate_eyebrow_movement(landmarks, left_brow_indices, right_brow_indices, baseline, movement_threshold):
    left_brow_y = np.mean([landmarks[i][1] for i in left_brow_indices])
    right_brow_y = np.mean([landmarks[i][1] for i in right_brow_indices])
    avg_brow_y = (left_brow_y + right_brow_y) / 2

    if baseline is None:
        baseline = avg_brow_y
        return "Neutral", baseline

    if avg_brow_y < baseline - movement_threshold:
        return "Eyebrows Raised", baseline
    else:
        baseline = 0.9 * baseline + 0.1 * avg_brow_y
        return "Neutral", baseline

# Function to send commands to Arduino
def send_command_to_arduino(arduino, command):
    if arduino:
        try:
            arduino.write(f"{command}\n".encode())
        except Exception as e:
            print(f"Error sending command: {e}")

# Function to overlay detection results
def overlay_text(frame, text, position, color=(0, 255, 0)):
    cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)

# Function to process face landmarks
def process_face_landmarks(face_landmarks, arduino, baseline_position, movement_threshold, frame):
    global blink_detected, claw_state
    
    LEFT_BROW = [46, 53, 52, 65, 55, 63, 105, 66, 107]
    RIGHT_BROW = [276, 283, 282, 295, 285, 293, 334, 296, 336]
    
    # Gaze detection
    left_iris = face_landmarks.landmark[468]
    right_iris = face_landmarks.landmark[473]
    left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133]]
    right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[263]]
    
    avg_gaze_ratio = (calculate_gaze_direction(left_iris, left_eye_corners) +
                      calculate_gaze_direction(right_iris, right_eye_corners)) / 2
    gaze_direction = "Center"
    if avg_gaze_ratio < 0.35:
        send_command_to_arduino(arduino, 'L')
        gaze_direction = "Left"
        gaze_color = (0, 0, 255)  # Red
    elif avg_gaze_ratio > 0.65:
        send_command_to_arduino(arduino, 'R')
        gaze_direction = "Right"
        gaze_color = (0, 0, 255)  # Red
    else:
        gaze_color = (0, 255, 0)  # Green (default)
    overlay_text(frame, f"Gaze: {gaze_direction}", (10, 30), gaze_color)

    # Blink detection and claw state
    left_eye_landmarks = [face_landmarks.landmark[133], face_landmarks.landmark[159], face_landmarks.landmark[145], face_landmarks.landmark[33]]
    right_eye_landmarks = [face_landmarks.landmark[362], face_landmarks.landmark[386], face_landmarks.landmark[374], face_landmarks.landmark[263]]
    current_blink = is_blinking(left_eye_landmarks) or is_blinking(right_eye_landmarks)
    if current_blink and not blink_detected:
        blink_detected = True
        claw_state = not claw_state
        send_command_to_arduino(arduino, 'OPEN' if claw_state else 'CLOSE')
    elif not current_blink:
        blink_detected = False
    claw_color = (0, 255, 0) if claw_state else (0, 0, 255)  # Green if open, Red if closed
    overlay_text(frame, f"Claw: {'Open' if claw_state else 'Closed'}", (10, 60), claw_color)

    # Eyebrow movement
    eyebrow_state, baseline_position = calculate_eyebrow_movement(
        [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark],
        LEFT_BROW, RIGHT_BROW, baseline_position, movement_threshold
    )
    send_command_to_arduino(arduino, "UP" if eyebrow_state == "Eyebrows Raised" else "DOWN")
    eyebrow_color = (0, 0, 255) if eyebrow_state == "Eyebrows Raised" else (0, 255, 0)  # Red if raised, Green if neutral
    overlay_text(frame, f"Eyebrows: {eyebrow_state}", (10, 90), eyebrow_color)

    # Mouth status
    mouth_status = detect_mouth_status(face_landmarks)
    send_command_to_arduino(arduino, 'EXTEND' if mouth_status == "Mouth Open" else 'RETRACT')
    mouth_color = (0, 0, 255) if mouth_status == "Mouth Open" else (0, 255, 0)  # Red if open, Green if closed
    overlay_text(frame, f"Mouth: {mouth_status}", (10, 120), mouth_color)

    return baseline_position

# Main function
def main():
    global blink_detected, claw_state
    blink_detected = False
    claw_state = False
    
    # Initialize resources
    arduino = initialize_serial('COM3')
    face_mesh = initialize_face_mesh()
    cap1 = cv2.VideoCapture(0)
    cap2 = cv2.VideoCapture(1)
    
    baseline_position = None
    movement_threshold = 0.01

    while cap1.isOpened() and cap2.isOpened():
        success1, frame1 = cap1.read()
        success2, frame2 = cap2.read()
        if not success1 or not success2:
            print("Error reading frames.")
            break
        
        # Flip the frames horizontally for better visualization
        frame1 = cv2.flip(frame1, 1)
        frame2 = cv2.flip(frame2, 1)
        
        # Convert frames to RGB for processing
        frame_rgb = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
        
        results = face_mesh.process(frame_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                baseline_position = process_face_landmarks(face_landmarks, arduino, baseline_position, movement_threshold, frame1)

        # Resize frames for better visualization
        frame1_resized = cv2.resize(frame1, (640, 400))
        frame2_resized = cv2.resize(frame2, (640, 400))

        # Stack frames verticallyq
        stacked_frame = np.vstack((frame1_resized, frame2_resized))

        # Show the stacked frames with overlayed text
        cv2.imshow("Feed", stacked_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap1.release()
    cap2.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Serial connection established.


D:\ANACONDA\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


### WITH EYE LANDMARKS

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import serial
import time

# Initialize serial communication with Arduino
def initialize_serial(port, baud_rate=9600):
    try:
        arduino = serial.Serial(port, baud_rate)
        time.sleep(2)  # Allow time for connection to initialize
        print("Serial connection established.")
        return arduino
    except Exception as e:
        print(f"Error initializing serial: {e}")
        return None

# Initialize Mediapipe FaceMesh
def initialize_face_mesh():
    mp_face_mesh = mp.solutions.face_mesh
    return mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Function to calculate gaze direction
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to detect blinking

def is_blinking(eye_landmarks, threshold=0.2):
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    blink_ratio = vertical_distance / horizontal_distance
    return blink_ratio < threshold

# Function to detect mouth status
def detect_mouth_status(landmarks):
    upper_lip_y = landmarks.landmark[61].y
    lower_lip_y = landmarks.landmark[17].y
    lip_distance = abs(upper_lip_y - lower_lip_y)
    
    left_eye = landmarks.landmark[33]
    right_eye = landmarks.landmark[263]
    eye_distance = abs(left_eye.x - right_eye.x)
    
    threshold = 0.025 + (eye_distance * 0.1)
    return "Mouth Open" if lip_distance > threshold else "Mouth Closed"

# Function to calculate eyebrow movement
def calculate_eyebrow_movement(landmarks, left_brow_indices, right_brow_indices, baseline, movement_threshold):
    left_brow_y = np.mean([landmarks[i][1] for i in left_brow_indices])
    right_brow_y = np.mean([landmarks[i][1] for i in right_brow_indices])
    avg_brow_y = (left_brow_y + right_brow_y) / 2

    if baseline is None:
        baseline = avg_brow_y
        return "Neutral", baseline

    if avg_brow_y < baseline - movement_threshold:
        return "Eyebrows Raised", baseline
    else:
        baseline = 0.9 * baseline + 0.1 * avg_brow_y
        return "Neutral", baseline

# Function to send commands to Arduino
def send_command_to_arduino(arduino, command):
    if arduino:
        try:
            arduino.write(f"{command}\n".encode())
        except Exception as e:
            print(f"Error sending command: {e}")

# Function to overlay detection results
def overlay_text(frame, text, position, color=(0, 255, 0)):
    cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)

# Function to process face landmarks
def process_face_landmarks(face_landmarks, arduino, baseline_position, movement_threshold, frame):
    global blink_detected, claw_state
    
    LEFT_BROW = [46, 53, 52, 65, 55, 63, 105, 66, 107]
    RIGHT_BROW = [276, 283, 282, 295, 285, 293, 334, 296, 336]
    
    # Gaze detection
    left_iris = face_landmarks.landmark[468]
    right_iris = face_landmarks.landmark[473]
    left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133]]
    right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[263]]
    
    avg_gaze_ratio = (calculate_gaze_direction(left_iris, left_eye_corners) +
                      calculate_gaze_direction(right_iris, right_eye_corners)) / 2
    gaze_direction = "Center"
    if avg_gaze_ratio < 0.35:
        send_command_to_arduino(arduino, 'L')
        gaze_direction = "Left"
        gaze_color = (0, 0, 255)  # Red
    elif avg_gaze_ratio > 0.65:
        send_command_to_arduino(arduino, 'R')
        gaze_direction = "Right"
        gaze_color = (0, 0, 255)  # Red
    else:
        gaze_color = (0, 255, 0)  # Green (default)
    overlay_text(frame, f"Gaze: {gaze_direction}", (10, 30), gaze_color)

    # Blink detection and claw state
    left_eye_landmarks = [face_landmarks.landmark[133], face_landmarks.landmark[159], face_landmarks.landmark[145], face_landmarks.landmark[33]]
    right_eye_landmarks = [face_landmarks.landmark[362], face_landmarks.landmark[386], face_landmarks.landmark[374], face_landmarks.landmark[263]]
    current_blink = is_blinking(left_eye_landmarks) or is_blinking(right_eye_landmarks)
    if current_blink and not blink_detected:
        blink_detected = True
        claw_state = not claw_state
        send_command_to_arduino(arduino, 'OPEN' if claw_state else 'CLOSE')
    elif not current_blink:
        blink_detected = False
    claw_color = (0, 255, 0) if claw_state else (0, 0, 255)  # Green if open, Red if closed
    overlay_text(frame, f"Claw: {'Open' if claw_state else 'Closed'}", (10, 60), claw_color)

    # Eyebrow movement
    eyebrow_state, baseline_position = calculate_eyebrow_movement(
        [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark],
        LEFT_BROW, RIGHT_BROW, baseline_position, movement_threshold
    )
    send_command_to_arduino(arduino, "UP" if eyebrow_state == "Eyebrows Raised" else "DOWN")
    eyebrow_color = (0, 0, 255) if eyebrow_state == "Eyebrows Raised" else (0, 255, 0)  # Red if raised, Green if neutral
    overlay_text(frame, f"Eyebrows: {eyebrow_state}", (10, 90), eyebrow_color)

    # Mouth status
    mouth_status = detect_mouth_status(face_landmarks)
    send_command_to_arduino(arduino, 'EXTEND' if mouth_status == "Mouth Open" else 'RETRACT')
    mouth_color = (0, 0, 255) if mouth_status == "Mouth Open" else (0, 255, 0)  # Red if open, Green if closed
    overlay_text(frame, f"Mouth: {mouth_status}", (10, 120), mouth_color)

    # Draw the pupil landmarks in red for both eyes
    left_pupil_landmark = face_landmarks.landmark[468]  # Left pupil center
    right_pupil_landmark = face_landmarks.landmark[473]  # Right pupil center
    height, width, _ = frame.shape
    
    left_pupil_x, left_pupil_y = int(left_pupil_landmark.x * width), int(left_pupil_landmark.y * height)
    right_pupil_x, right_pupil_y = int(right_pupil_landmark.x * width), int(right_pupil_landmark.y * height)
    
    # Red dot for both pupils
    cv2.circle(frame, (left_pupil_x, left_pupil_y), 3, (0, 0, 255), -1)
    cv2.circle(frame, (right_pupil_x, right_pupil_y), 3, (0, 0, 255), -1)

    return baseline_position

# Main function
def main():
    global blink_detected, claw_state
    blink_detected = False
    claw_state = False
    
    # Initialize resources
    arduino = initialize_serial('COM4')
    face_mesh = initialize_face_mesh()
    cap1 = cv2.VideoCapture(0)
    cap2 = cv2.VideoCapture(1)
    
    baseline_position = None
    movement_threshold = 0.01

    while cap1.isOpened() and cap2.isOpened():
        success1, frame1 = cap1.read()
        success2, frame2 = cap2.read()
        if not success1 or not success2:
            print("Error reading frames.")
            break
        
        # Flip the frames horizontally for better visualization
        frame1 = cv2.flip(frame1, 1)
        frame2 = cv2.flip(frame2, 1)
        
        # Convert frames to RGB for processing
        frame_rgb = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
        
        results = face_mesh.process(frame_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                baseline_position = process_face_landmarks(face_landmarks, arduino, baseline_position, movement_threshold, frame1)

        # Resize frames for better visualization
        frame1_resized = cv2.resize(frame1, (640, 400))
        frame2_resized = cv2.resize(frame2, (640, 400))

        # Stack frames vertically
        stacked_frame = np.vstack((frame1_resized, frame2_resized))

        # Show the stacked frames with overlayed text
        cv2.imshow("Feed", stacked_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap1.release()
    cap2.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Serial connection established.
